In [1]:
import pandas as pd
import numpy as np
df1 = pd.read_csv("IMDb movies copy.csv")
df2 = pd.read_csv("IMDb ratings copy.csv")
df3 = pd.read_csv("movies_metadata copy.csv")
df4 = pd.read_csv("MoviesOnStreamingPlatforms_updated copy.csv")

/Users/McBride/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/Users/McBride/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [2]:
#match streaming ID key to IMDb ID key

In [3]:
#filter datasets for relevent columns
df1=df1[['imdb_title_id','title','year','duration']]
df3=df3[['imdb_id','title','original_title','budget','revenue','runtime','adult']]
df4=df4.drop(columns=['Unnamed: 0','Type'])

In [4]:
#merge imbd datasets on imbd_id
merge = pd.merge(df1, df3, left_on='imdb_title_id', right_on='imdb_id', how='inner')
#make title columns lowercase and remove punctuation
merge['title_x']=merge['title_x'].str.lower().str.replace('[^\w\s]','')
merge['title_y']=merge['title_y'].str.lower().str.replace('[^\w\s]','')
merge['original_title']=merge['original_title'].str.lower().str.replace('[^\w\s]','')
df4['Title']=df4['Title'].str.lower().str.replace('[^\w\s]','')

In [5]:
#merge imdb data with platform data on all title columns
merge1 = pd.merge(merge,df4, left_on=['title_y'], right_on=['Title'], how='inner')
merge2 = pd.merge(merge,df4, left_on=['title_x'], right_on=['Title'], how='inner')
merge3 = pd.merge(merge,df4, left_on=['original_title'], right_on=['Title'], how='inner')
#concatenate three databases
concat = pd.concat([merge1,merge2,merge3])
#delete duplicate ID values
concat = concat.drop_duplicates(subset='ID', keep='first')
#delete duplicate imdb values
concat = concat.drop_duplicates(subset='imdb_id', keep='first')

In [6]:
#SQL Table key match
ids = concat[['imdb_title_id','ID']]
ids.columns = ids.columns.str.lower()
ids.to_csv('ids.csv', index=False)
ids.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5421 entries, 0 to 4547
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   imdb_title_id  5421 non-null   object
 1   id             5421 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 127.1+ KB


In [7]:
#SQL Tables from imdb movie dataset

In [8]:
#main table
df1['title']=df1['title'].str.lower().str.replace('[^\w\s]','').astype('string')
df1['year']=df1['year'].astype('string')
df1['year']=df1['year'].str.replace('TV Movie 2019', '2019')
df1['year']=df1['year'].fillna('0')
df1['year']=df1['year'].astype('int')
df1.to_csv('imdb_movies.csv', index=False)
df1.info()
#df1['year'].unique()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85855 entries, 0 to 85854
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   imdb_title_id  85855 non-null  object
 1   title          85855 non-null  string
 2   year           85855 non-null  int64 
 3   duration       85855 non-null  int64 
dtypes: int64(2), object(1), string(1)
memory usage: 2.6+ MB


In [9]:
#SQL Tables from ratings dataset

In [10]:
#imdb ratings
imdb_ratings= df2[['imdb_title_id','weighted_average_vote','total_votes','mean_vote','median_vote','top1000_voters_rating','top1000_voters_votes']]
imdb_ratings = imdb_ratings.rename(columns={'weighted_average_vote':'weighted_avg','total_votes':'total_votes_count','top1000_voters_votes':'top1000_voters_count'})
imdb_ratings.to_csv('imdb_ratings.csv', index=False)
imdb_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85855 entries, 0 to 85854
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   imdb_title_id          85855 non-null  object 
 1   weighted_avg           85855 non-null  float64
 2   total_votes_count      85855 non-null  int64  
 3   mean_vote              85855 non-null  float64
 4   median_vote            85855 non-null  float64
 5   top1000_voters_rating  85176 non-null  float64
 6   top1000_voters_count   85176 non-null  float64
dtypes: float64(5), int64(1), object(1)
memory usage: 4.6+ MB


In [11]:
#imdb votes by location
loc_votes=df2[['imdb_title_id','us_voters_votes','non_us_voters_votes']]
loc_votes=loc_votes.melt(id_vars=['imdb_title_id'],var_name='location',value_name='vote_count')
loc_votes['location']=loc_votes['location'].replace(['us_voters_votes','non_us_voters_votes'],['us','non_us'])


loc_rating=df2[['imdb_title_id','us_voters_rating','non_us_voters_rating']]
loc_rating=loc_rating.melt(id_vars=['imdb_title_id'],var_name='location',value_name='rating')
loc_rating['location']=loc_rating['location'].replace(['us_voters_rating','non_us_voters_rating'],['us','non_us'])

imdb_votes_by_location = pd.merge(loc_votes, loc_rating, how='inner')
imdb_votes_by_location.to_csv('imdb_votes_by_location.csv', index=False)
imdb_votes_by_location.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 171710 entries, 0 to 171709
Data columns (total 4 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   imdb_title_id  171710 non-null  object 
 1   location       171710 non-null  object 
 2   vote_count     171500 non-null  float64
 3   rating         171500 non-null  float64
dtypes: float64(2), object(2)
memory usage: 6.6+ MB


In [12]:
#imdb votes by gender
gender1=df2.iloc[:,[0,15,17,19,21,23,25,27,29,31,33,35,37,39,41]]
gender1=gender1.melt(id_vars='imdb_title_id', var_name='temp', value_name='rating')
gender1['gender']=gender1['temp'].str.split('_').str[0]
gender1['ages']=gender1['temp'].str.split('_').str[1]
gender1 = gender1.drop(columns=['temp'])

gender2=df2.iloc[:,[0,16,18,20,22,24,26,28,30,32,34,36,38,40,42]]
gender2=gender2.melt(id_vars='imdb_title_id', var_name='temp', value_name='vote_count')
gender2['gender']=gender2['temp'].str.split('_').str[0]
gender2['ages']=gender2['temp'].str.split('_').str[1]
gender2 = gender2.drop(columns=['temp'])

imdb_votes_by_gender_age = pd.merge(gender1,gender2)
imdb_votes_by_gender_age['ages']=imdb_votes_by_gender_age['ages'].replace(['0age','18age','30age','45age'],['under 18','18+','30+','45+'])
imdb_votes_by_gender_age.to_csv('imdb_votes_by_gender_age.csv', index=False)
imdb_votes_by_gender_age.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1201970 entries, 0 to 1201969
Data columns (total 5 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   imdb_title_id  1201970 non-null  object 
 1   rating         1014573 non-null  float64
 2   gender         1201970 non-null  object 
 3   ages           1201970 non-null  object 
 4   vote_count     1014573 non-null  float64
dtypes: float64(2), object(3)
memory usage: 55.0+ MB


In [13]:
imdb_votes_by_gender_age['ages'].unique()

array(['under 18', '18+', '30+', '45+', 'allages'], dtype=object)

In [14]:
imdb_votes_by_gender_age['gender'].unique()

array(['allgenders', 'males', 'females'], dtype=object)

In [15]:
#imdb votes by rating
score=df2.iloc[:,[0,5,6,7,8,9,10,11,12,13,14]]
score=score.melt(id_vars='imdb_title_id', var_name='temp', value_name='vote_count')
score['score']=score['temp'].str.split('_').str[1]
score = score.drop(columns=['temp'])
score.to_csv('imdb_vote_count_by_score.csv', index=False)
score.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 858550 entries, 0 to 858549
Data columns (total 3 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   imdb_title_id  858550 non-null  object
 1   vote_count     858550 non-null  int64 
 2   score          858550 non-null  object
dtypes: int64(1), object(2)
memory usage: 19.7+ MB


In [16]:
#SQL Tables from entire Platform dataset

In [17]:
#genres
subset1 = df4[['ID','Genres']]
genres=subset1.assign(Genres=subset1.Genres.str.split(',')).explode('Genres')
genres.columns = genres.columns.str.lower()
genres.to_csv('genres.csv', index=False)
genres.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39373 entries, 0 to 16743
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      39373 non-null  int64 
 1   genres  39098 non-null  object
dtypes: int64(1), object(1)
memory usage: 922.8+ KB


In [18]:
#directors
subset2 = df4[['ID','Directors']]
directors=subset2.assign(Directors=subset2.Directors.str.split(',')).explode('Directors')
directors.columns = directors.columns.str.lower()
directors.to_csv('directors.csv', index=False)
directors.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18924 entries, 0 to 16743
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         18924 non-null  int64 
 1   directors  18198 non-null  object
dtypes: int64(1), object(1)
memory usage: 443.5+ KB


In [19]:
#country
subset3 = df4[['ID','Country']]
countries=subset3.assign(Country=subset3.Country.str.split(',')).explode('Country')
countries.columns = countries.columns.str.lower()
countries.to_csv('countries.csv', index=False)
countries.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21134 entries, 0 to 16743
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       21134 non-null  int64 
 1   country  20699 non-null  object
dtypes: int64(1), object(1)
memory usage: 495.3+ KB


In [20]:
#language
subset4 = df4[['ID','Language']]
languages=subset4.assign(Language=subset4.Language.str.split(',')).explode('Language')
languages.columns = languages.columns.str.lower()
languages.to_csv('languages.csv', index=False)
languages.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20956 entries, 0 to 16743
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        20956 non-null  int64 
 1   language  20357 non-null  object
dtypes: int64(1), object(1)
memory usage: 491.2+ KB


In [21]:
#streaming platform table of movies
movies=df4[['ID','Title','Year','Runtime','Age','IMDb','Rotten Tomatoes']]
movies['Rotten Tomatoes']=movies['Rotten Tomatoes'].str.strip('%').astype('float64').div(100)
movies.columns = movies.columns.str.lower()
movies=movies.rename(columns={'imdb':'imdb_rating','rotten tomatoes':'rotten_tomatoes_rating','age':'rated_ages_above'})
movies.to_csv('movies.csv', index=False)
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16744 entries, 0 to 16743
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      16744 non-null  int64  
 1   title                   16744 non-null  object 
 2   year                    16744 non-null  int64  
 3   runtime                 16152 non-null  float64
 4   rated_ages_above        7354 non-null   object 
 5   imdb_rating             16173 non-null  float64
 6   rotten_tomatoes_rating  5158 non-null   float64
dtypes: float64(3), int64(2), object(2)
memory usage: 915.8+ KB


<ipython-input-21-f2fcb44b13a4>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies['Rotten Tomatoes']=movies['Rotten Tomatoes'].str.strip('%').astype('float64').div(100)


In [22]:
#platforms
subset5 = df4[['ID','Netflix','Hulu','Prime Video','Disney+']]
subset5['Netflix']=subset5['Netflix'].replace(1,'Netflix')
subset5['Hulu']=subset5['Hulu'].replace(1,'Hulu')
subset5['Prime Video']=subset5['Prime Video'].replace(1,'Prime Video')
subset5['Disney+']=subset5['Disney+'].replace(1,'Disney+')
platforms=subset5.melt(id_vars=['ID'],var_name='Platforms',value_name='Platform')
platforms=platforms.drop(columns=['Platforms'])
platforms=platforms[platforms.Platform != 0]
platforms.columns = platforms.columns.str.lower()
platforms.to_csv('platforms.csv', index=False)
platforms.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17381 entries, 0 to 66975
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        17381 non-null  int64 
 1   platform  17381 non-null  object
dtypes: int64(1), object(1)
memory usage: 407.4+ KB


<ipython-input-22-ce5db6c59b1b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset5['Netflix']=subset5['Netflix'].replace(1,'Netflix')
<ipython-input-22-ce5db6c59b1b>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset5['Hulu']=subset5['Hulu'].replace(1,'Hulu')
<ipython-input-22-ce5db6c59b1b>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyda